# EDA

## Libaries

In [ ]:
import pandas as pd
import numpy as np

#visualisation
import matplotlib.pyplot as plt
import seaborn as sns

#nltk
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
#import für eda_1 - preprozessing dataframe
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# default setting for lemmatizer and stopwords
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

# remove stop words
nltk.download('stopwords')
my_stopwords = set(stopwords.words('english'))

##libaries for part od speech

In [ ]:
import spacy
import csv

In [ ]:
'''# SPACY  deutsch!!!
!python -m spacy download de_core_news_sm
''''

In [ ]:
# to spacy---- separate, while import de... after !python.....
import spacy
import de_core_news_sm
import en_core_web_sm

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
from nltk import pos_tag
from collections import Counter
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

from nltk import pos_tag
from nltk import FreqDist
from nltk.tokenize import PunktSentenceTokenizer

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


##Functions

In [ ]:
# from folder data
# import data.get_data.process_question(row)
def process_question(row):
   '''join row to text''' 
   text =  " ".join(row.split(" ")[1:])
   return text

In [86]:
def get_plot_length_text(df, title, diagram):
    '''
    creates a histogram 
    that shows the distribution of the number of tokens 
    per line for the data set to be examined
    
    Parameters
    ----------
    df:
        series; to examined text
    title: 
        string; title for Plot
    diagram: 
        string: name of the diagram
    
    Returns
    -------
    Vizualisationplot Distribution
    '''
    plt.hist(df.apply(lambda text: len(text.split())))
    plt.xlabel('number of token')
    plt.ylabel('frequency')
    plt.title(title)
    plt.savefig(path_plot + diagram + '.png')
    plt.show()
    
    avg = round(df.apply(lambda text: len(text.split())).mean())
    maxi = max(list(df.apply(lambda text: len(text.split()))))
    mini = min(list(df.apply(lambda text: len(text.split()))))
    
    print(f'The texts have a mean length of {avg} tokens.')
    print(f'The longest text has {maxi} tokens, the shortest has {mini} tokens.')

In [ ]:
def corpus_func(df):
    '''
    create a textcorpus from pd.series, df['column']

    Parameters
    ----------
    text, string

    Returns
    ------_
    concatinated string with marker --XXX-- as selector
    '''
    return " XXX ".join(text for text in df)

def split_corpus_func(corpus):
    '''
    create a column from text corpus with marker ' XXX '
    as selector

    Parameters
    ----------
    text, string

    Returns
    ------_
    list of text 
    '''
    list = lambda x: x.split(' XXX ')
    return list

def preprocess_dataframe(df):
    '''
    create new columns in the data frame
    new_column: 'text' => cleaned stopwords (english)
                'text_clean' => regex, lowercase
                'text_lemma' => lemmetized
    param: df
    returns: df with new columns
    '''

    corpus = corpus_func(df['question'])
    text_corpus = stopword_text(corpus)
    df['text'] = split_corpus_func(corpus)
    clean_corpus = clean_text(text_corpus)
    df['text_clean'] = split_corpus_func(clean_corpus)
    lemma = lem_text(clean_corpus)
    df['text_lemma'] = split_corpus_func(clean_corpus)
    return df


In [ ]:
#import utils.text_manipulation

def clean_text(text):
    """
    Regex cleaning of the text. Filters everything except alphanumerical and '.
    Return is turned into lower case

    Parameters
    ----------
    text : string
        text to be cleaned

    Returns
    -------
    string
        lower case regex cleaned text

    """
    text = text.replace("´", "'")
    
    text = text.replace("'s", " ")

    digi_punct = "[^a-zA-Z.1234567890#' ]"
    text = re.sub(digi_punct, " ", text)
    text = " ".join(text.split())
    text = text.lower()
    return text

def stopword_text(text):
    """
    Remove all words in the text that are in the stopword list
     
    Parameters
    ----------
    text : string
    
    Returns
    -------
    string
        text only stopwords
     
    """

    return " ".join([word for word in text.split() if word not in my_stopwords])

def lem_text(text):
    """
    Group the different inflected forms of a word so they can be analysed as 
    a single item
     
    Parameters
    ----------
    text : string
    
    Returns
    -------
    string
        text with lemmas
    """

    lem_sentence = text.split()
    for i, word in enumerate(text.split()):
        for pos in "n", "v", "a", "r":
            lem = lemmatizer.lemmatize(word, pos=pos)
            if lem != word:
                lem_sentence[i] = lem
                break
            else:
                lem_sentence[i] = word
    return " ".join(lem_sentence)


# Load data

In [ ]:
# data
path_class_def  = 'https://cogcomp.seas.upenn.edu/Data/QA/QC/definition.html'
path_train_data = 'https://cogcomp.seas.upenn.edu/Data/QA/QC/train_5500.label'
path_test_data  = 'https://cogcomp.seas.upenn.edu/Data/QA/QC/TREC_10.label'

In [ ]:
#load
train_df = pd.read_table(path_train_data, encoding = "ISO-8859-1", header=None)
train_df.columns = ["raw"]
train_df['category'] = train_df.apply (lambda row: row["raw"].split(":")[0], axis=1)
train_df['subcategory'] = train_df.apply (lambda row: row["raw"].split(" ")[0].split(":")[1], axis=1)
train_df['question'] = train_df.apply (lambda row: process_question(row["raw"]), axis=1)

train_df.head(2)

,raw,category,subcategory,question
0,DESC:manner How did serfdom develop in and the...,DESC,manner,How did serfdom develop in and then leave Russ...
1,ENTY:cremat What films featured the character ...,ENTY,cremat,What films featured the character Popeye Doyle ?


In [ ]:
test_df = pd.read_table(path_test_data, encoding = "ISO-8859-1", header=None)
test_df.columns = ["raw"]
test_df['category'] = train_df.apply (lambda row: row["raw"].split(":")[0], axis=1)
test_df['subcategory'] = train_df.apply (lambda row: row["raw"].split(" ")[0].split(":")[1], axis=1)
test_df['question'] = train_df.apply (lambda row: process_question(row["raw"]), axis=1)

test_df.head(2)

,raw,category,subcategory,question
0,NUM:dist How far is it from Denver to Aspen ?,DESC,manner,How did serfdom develop in and then leave Russ...
1,"LOC:city What county is Modesto , California in ?",ENTY,cremat,What films featured the character Popeye Doyle ?


#corpus

In [ ]:
# corpus from df_train['question'] 
corpus = corpus_func(train_df['question'])
print(corpus[:500])

How did serfdom develop in and then leave Russia ? XXX What films featured the character Popeye Doyle ? XXX How can I find a list of celebrities ' real names ? XXX What fowl grabs the spotlight after the Chinese Year of the Monkey ? XXX What is the full form of .com ? XXX What contemptible scoundrel stole the cork from my lunch ? XXX What team did baseball 's St. Louis Browns become ? XXX What is the oldest profession ? XXX What are liver enzymes ? XXX Name the scar-faced bounty hunter of The Ol


In [ ]:
#corpus from df_test['question']
corpus_test = corpus_func(test_df['question'])
print(corpus_test[:500])

How did serfdom develop in and then leave Russia ? XXX What films featured the character Popeye Doyle ? XXX How can I find a list of celebrities ' real names ? XXX What fowl grabs the spotlight after the Chinese Year of the Monkey ? XXX What is the full form of .com ? XXX What contemptible scoundrel stole the cork from my lunch ? XXX What team did baseball 's St. Louis Browns become ? XXX What is the oldest profession ? XXX What are liver enzymes ? XXX Name the scar-faced bounty hunter of The Ol


In [ ]:
text = corpus

In [ ]:
#sample
text_sample = process_question(train_df['raw'][10])


#Part of speech tags

    

 

In [ ]:
# Split the data into individual tokens.
text_splitted = text.split()


# or:
from nltk.tokenize import word_tokenize 
word_tokens = word_tokenize(text) 

print('split:\t',text_splitted[:100],'\ntokenize:',word_tokens[:100])

split:	 ['How', 'did', 'serfdom', 'develop', 'in', 'and', 'then', 'leave', 'Russia', '?', 'XXX', 'What', 'films', 'featured', 'the', 'character', 'Popeye', 'Doyle', '?', 'XXX', 'How', 'can', 'I', 'find', 'a', 'list', 'of', 'celebrities', "'", 'real', 'names', '?', 'XXX', 'What', 'fowl', 'grabs', 'the', 'spotlight', 'after', 'the', 'Chinese', 'Year', 'of', 'the', 'Monkey', '?', 'XXX', 'What', 'is', 'the', 'full', 'form', 'of', '.com', '?', 'XXX', 'What', 'contemptible', 'scoundrel', 'stole', 'the', 'cork', 'from', 'my', 'lunch', '?', 'XXX', 'What', 'team', 'did', 'baseball', "'s", 'St.', 'Louis', 'Browns', 'become', '?', 'XXX', 'What', 'is', 'the', 'oldest', 'profession', '?', 'XXX', 'What', 'are', 'liver', 'enzymes', '?', 'XXX', 'Name', 'the', 'scar-faced', 'bounty', 'hunter', 'of', 'The', 'Old', 'West'] 
tokenize: ['How', 'did', 'serfdom', 'develop', 'in', 'and', 'then', 'leave', 'Russia', '?', 'XXX', 'What', 'films', 'featured', 'the', 'character', 'Popeye', 'Doyle', '?', 'XXX', 'How

In [ ]:
# Tag each token into a specific part of speech 
word_tagged = nltk.pos_tag(word_tokens)

print('tagged:\t',word_tagged[:10])

tagged:	 [('How', 'WRB'), ('did', 'VBD'), ('serfdom', 'VB'), ('develop', 'VB'), ('in', 'IN'), ('and', 'CC'), ('then', 'RB'), ('leave', 'VB'), ('Russia', 'NNP'), ('?', '.')]


In [ ]:
# Print out the frequency for each part of speech. (i.e. 2 NN's, 100 VBG's, etc.)
# To see the acronym definitions for each part of speech, look at the following link: https://pythonprogramming.net/part-of-speech-tagging-nltk-tutorial/

from collections import Counter  

word_count = sorted(Counter(word_tagged).items(), key = lambda kv:( kv[1], kv[0]), reverse=True)

#or with nltk FreqDist
from nltk import FreqDist   
fdist = FreqDist(word_tagged)
first20_count = fdist.most_common(10)


print(f'word_count:{word_count}\n len word_count: {len(word_count)}')
print(f'first20_count :{first20_count }')

word_count:[(('?', '.'), 5352), (('the', 'DT'), 3612), (('XXX', 'VB'), 3430), (('What', 'WP'), 3246), (('is', 'VBZ'), 1679), (('of', 'IN'), 1546), (('in', 'IN'), 1133), (('a', 'DT'), 1010), (('XXX', 'NN'), 921), (('XXX', 'NNP'), 781), (('was', 'VBD'), 652), (('to', 'TO'), 604), ((',', ','), 571), (("'s", 'POS'), 537), (('How', 'NNP'), 460), (('are', 'VBP'), 453), (('for', 'IN'), 450), (('and', 'CC'), 425), (("''", "''"), 401), (('``', '``'), 399), (('does', 'VBZ'), 390), (('did', 'VBD'), 388), (('Who', 'WP'), 388), (('many', 'JJ'), 332), (('on', 'IN'), 306), (('How', 'WRB'), 300), (('name', 'NN'), 237), (('Where', 'WRB'), 230), (('I', 'PRP'), 209), (('can', 'MD'), 190), (('you', 'PRP'), 186), (("'s", 'VBZ'), 184), (('Who', 'NNP'), 173), (('first', 'JJ'), 172), (('do', 'VB'), 160), (('The', 'DT'), 160), (('do', 'VBP'), 156), (('from', 'IN'), 154), (('.', '.'), 143), (('by', 'IN'), 135), (('U.S.', 'NNP'), 134), (('country', 'NN'), 132), (('When', 'WRB'), 132), (('what', 'WP'), 127), (('a

Text: The original word text.
Lemma: The base form of the word.
POS: The simple UPOS part-of-speech tag.
Tag: The detailed part-of-speech tag.
Dep: Syntactic dependency, i.e. the relation between tokens.
Shape: The word shape – capitalization, punctuation, digits.
is alpha: Is the token an alpha character?
is stop: Is the token part of a stop list, i.e. the most common words of the language?

In [95]:
doc = nlp(text_sample)

print('token','-', 'lemma_', '-', 'lower_''-', 'pos_','-', 'tag_-', 'dep_','-', 'sentiment','-','is_alpha','-', 'is_stop')

for token in doc:
    #any token_attributs; more: https://spacy.io/api/token#attributes
    
    print('------------------------------------------------------------------------\n')   
    print(token.text,'-', token.lemma_, '-', token.lower_,'-', token.pos_,'-', token.tag_,'-',
          token.dep_,'-', token.sentiment,'-', token.is_alpha,'-', token.is_stop)

'''
Text: The original word text.
Lemma: The base form of the word.
POS: The simple UPOS part-of-speech tag.
Tag: The detailed part-of-speech tag.
Dep: Syntactic dependency, i.e. the relation between tokens.
Shape: The word shape – capitalization, punctuation, digits.
is alpha: Is the token an alpha character?
is stop: Is the token part of a stop list, i.e. the most common words of the language?'''

token - lemma_ - lower_- pos_ - tag_- dep_ - sentiment - is_alpha - is_stop
------------------------------------------------------------------------

When - when - when - ADV - WRB - advmod - 0.0 - True - True
------------------------------------------------------------------------

was - be - was - AUX - VBD - auxpass - 0.0 - True - True
------------------------------------------------------------------------

Ozzy - Ozzy - ozzy - PROPN - NNP - compound - 0.0 - True - False
------------------------------------------------------------------------

Osbourne - Osbourne - osbourne - PROPN - NNP - nsubj - 0.0 - True - False
------------------------------------------------------------------------

born - bear - born - VERB - VBN - ROOT - 0.0 - True - False
------------------------------------------------------------------------

? - ? - ? - PUNCT - . - punct - 0.0 - False - False


'\nText: The original word text.\nLemma: The base form of the word.\nPOS: The simple UPOS part-of-speech tag.\nTag: The detailed part-of-speech tag.\nDep: Syntactic dependency, i.e. the relation between tokens.\nShape: The word shape – capitalization, punctuation, digits.\nis alpha: Is the token an alpha character?\nis stop: Is the token part of a stop list, i.e. the most common words of the language?'

#hier weiter

In [ ]:
# Print out the entire spacy text.   --- split by 300 ...to long for display
print(doc[:300])

When was Ozzy Osbourne born ?


In [ ]:
# check doc is tagged?
print(doc.is_tagged)

True


In [ ]:
#token_tags + explain
print([(token.text, token.tag_, spacy.explain(token.tag_)) for token in doc])

[('When', 'WRB', 'wh-adverb'), ('was', 'VBD', 'verb, past tense'), ('Ozzy', 'NNP', 'noun, proper singular'), ('Osbourne', 'NNP', 'noun, proper singular'), ('born', 'VBN', 'verb, past participle'), ('?', '.', 'punctuation mark, sentence closer')]


In [ ]:
#_entities + label
print([(ent.text, ent.label_) for ent in doc.ents])

[('Ozzy Osbourne', 'PERSON')]


In [ ]:
#token_dep_ + explain
print([(token.text, token.dep_, spacy.explain(token.dep_)) for token in doc])

[('When', 'advmod', 'adverbial modifier'), ('was', 'auxpass', 'auxiliary (passive)'), ('Ozzy', 'compound', 'compound'), ('Osbourne', 'nsubj', 'nominal subject'), ('born', 'ROOT', None), ('?', 'punct', 'punctuation')]


spaCy add on:
doc, any attributes, chunk

In [ ]:
#any token_attributs; more: https://spacy.io/api/token#attributes
print('token','-', 'lemma_', '-', 'lower_''-', 'pos_','-', 'tag_-', 'dep_','-', 'sentiment','-','is_alpha','-', 'is_stop')
print('------------------------------------------------------------------------')   
for i, token in zip(range (10), doc):
    print(i, token.text,'-', token.lemma_, '-', token.lower_,'-', token.pos_,'-', token.tag_,'-',
          token.dep_,'-', token.sentiment,'-', token.is_alpha,'-', token.is_stop)

token - lemma_ - lower_- pos_ - tag_- dep_ - sentiment - is_alpha - is_stop
------------------------------------------------------------------------
0 When - when - when - ADV - WRB - advmod - 0.0 - True - True
1 was - be - was - AUX - VBD - auxpass - 0.0 - True - True
2 Ozzy - Ozzy - ozzy - PROPN - NNP - compound - 0.0 - True - False
3 Osbourne - Osbourne - osbourne - PROPN - NNP - nsubj - 0.0 - True - False
4 born - bear - born - VERB - VBN - ROOT - 0.0 - True - False
5 ? - ? - ? - PUNCT - . - punct - 0.0 - False - False


In [ ]:
#„Basis-Nomen-Phrasen“ - flache Phrasen, deren Kopf ein Nomen enthält
for chunk in doc[0:10].noun_chunks:
    print(chunk.text, chunk.root.text, chunk.root.dep_,
            chunk.root.head.text)

Ozzy Osbourne Osbourne nsubj born


In [ ]:
# For the first 100 words, print out the text, tag, and explanation of the tag.

# If you want to format things better, try using f strings! For example, look at the following:
# for number in range(0, 100):
#   print(f"Original Number: {number}, {' ':{10}} Original Number times 2: {number * 10}") 

i = 0
for i, token in zip(range(99),doc):
    print(f"{token}\t{' ':{7}}\t{token.tag_}\t\t{' ':{8}}{spacy.explain(token.tag_)}")

# {' ':{10} or \t ---> it won't be very nice

When	       	WRB		        wh-adverb
was	       	VBD		        verb, past tense
Ozzy	       	NNP		        noun, proper singular
Osbourne	       	NNP		        noun, proper singular
born	       	VBN		        verb, past participle
?	       	.		        punctuation mark, sentence closer


In [ ]:
#Count the frequencies of a given attribute.
from spacy.attrs import ORTH

print(doc.count_by(ORTH))
print(len(doc.count_by(ORTH)))
# and now???

{10109588199364727116: 1, 9921686513378912864: 1, 10497583950216086740: 1, 14167773886254661413: 1, 14611628419808709137: 1, 8205403955989537350: 1}
6


In [ ]:
#other way

dict_tags={}
for token in doc:
  #print(token,token.tag_)
  try:
    dict_tags[token.tag_].append(token.text)
  except:
    dict_tags[token.tag_]=[token.text]

print(dict_tags)

{'WRB': ['When'], 'VBD': ['was'], 'NNP': ['Ozzy', 'Osbourne'], 'VBN': ['born'], '.': ['?']}


In [ ]:
tag_labels = []
for key in doc:
    while token.head != token:
        tag_labels.append(token.tag_)
        token = token.head
print(tag_labels)

['.']


In [ ]:
tags_count=[]
for value in dict_tags.values():
    tags_count.append(len(value))
print(tags_count) 

[1, 1, 2, 1, 1]


In [ ]:
# Visualize the POS tags of the first sentence in the original text file.

#sentence
sents = list(doc.sents)
print(len(sents))
sents[0]

print(sents[0])

from spacy import displacy 
displacy.render(sents[0],style="dep" ,jupyter=True, options = {'distance' : 100})


1
When was Ozzy Osbourne born ?


In [ ]:
from spacy import displacy
displacy.render(sents[0],style="ent" ,jupyter=True)

reiner klau                                              
https://shirishkadam.com/2017/07/03/nlp-question-classification-using-support-vector-machines-spacyscikit-learnpandas/

In [ ]:
# import spacy
# import csv
# 
# clean_old_data()
# en_nlp = spacy.load("en_core_web_md")

In [ ]:
def process_question (question, qclass, en_nlp): 
    en_doc = en_nlp (u '' + question) 
    sent_list = list (en_doc.sents) 
    sent = sent_list [0] 
    wh_bi_gram = [] 
    root_token = "" 
    wh_pos = "" 
    wh_nbor_pos = "" 
    wh_word = "" 
    für gesendetes Token: 
        wenn token.tag_ == "WDT" oder token.tag_ == "WP" oder token.tag_ == "WP $" oder token.tag_ == "WRB": 
            wh_pos = token .tag_ 
            wh_word = token.text 
            wh_bi_gram.append (token.text) 
            wh_bi_gram.append (str (en_doc [token.i + 1])) 
            wh_nbor_pos = en_doc [token.i + 1].tag_ 
        if token.dep_ == "ROOT":
            root_token = token.tag_ 

    write_each_record_to_csv (wh_pos, wh_word, wh_bi_gram, wh_nbor_pos, root_token)

process_question (df_train['question'][50])